# Homework 1

1.1

Introspection is a subjective process for humans, and it is only capable of informing so much. In Cognitive Psychology, we are learning that cognition is broadly created from bottom-up and top-down processes, simultaneously handling and filtering many stimuli before they reach consciousness. These automatic components of cognition would be undetectable to introspection and thus cannot inform efforts in AI problem-solving. An example can be that in solving a problem, humans (subconsciously) choose to attend to certain features while ignoring others. If only introspection is used in abstracting how we came to the solution, AI would miss out on how we pre-filter the information to focus on the useful features and be lost in a sea of information. Another broad idea in cognitive psychology is how we virtually always impose prior knowledge and experience on our current situation. This top-down process would be pretty hard to capture and translate to a machine using introspection, since computers need to be programs to glean information from past experiences and "learn". So personally I think that we need something that is more objective and well-defined than just introspection to make problem-solving practical for machines. Cognitive tests and experienments can zero-in on what happens under the hood in human thought processes. Nonetheless, introspection tells us how we prioritize certain information and make general judgements, these principles can serve as a foundation to build AI algorithms heuristics.

1.2
I formulated traveling salesman as a local search problem. Each city is coded as an integer, and the distances between any two cities (random integer between 0 and 20) is stored as a key-value pair in a dictionary. The problem starts with a initial state, which is a random complete path. From here, the problem allows for actions that swap the order of visit between any two cities excluding the first and last in the circuit. It is required that any state starts at city 0, hits all cities, and returns to 0. At any state, The total travelled distance for the current state is calculated by adding up the pair-wise distances in the circuit. Pair-wise distances are found in the map, and it is assumed that distances are symmetric. We'd want to optimize and find the shortest possible path, so actions that decreases the total distance of the current state are defined to have a higher value.

In [ ]:
print('Here\'s the problem formulation')

Here's the problem formulation


In [ ]:
## TSP problem formulation
from tools.aima.search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
from threading import Timer
import math
import time

debug = 0
class TravelingSalesman(Problem):
    """
    initial: a random complete city circuit.
    map: a dictionary containing the distances between any two cities in the circuit.
    """
    def __init__(self, numCities, initial, map):
        self.num = numCities
        self.initial = initial
        self.map = map

    # actions returns a list of pairs of cities to swap in the current state
    def actions(self, state):
        """
        :param state: the current state containing a circuit
        :return: actions, a list containing tuples of pairs of cities to swap order in the circuit. The list contains all possible swaps
        """
        actions = []
        lastCity = max(state)
        i = 1
        while i < (len(state)-2) :
            currentCity = state[i]
            for selectSwap in range(1, lastCity+1):
                if currentCity != selectSwap:
                    actions.append([currentCity, selectSwap])
            i += 1
        return actions

    def result(self, state, action):
        """
        :param state: current state in the tsp
        :param action: a selected action (pair of cities to swap)
        :return: the new state resulting from swapping the selected pair of cities to the given state
        """
        new_state = state[:]
        swap_1 = state.index(action[0])
        swap_2 = state.index(action[1])
        new_state[swap_1] = action[1]
        new_state[swap_2] = action[0]
        return new_state

    def value(self, state):
        """
        :param state: the current city circuit
        :return: the sum of the travelling distances in the given state, negated in order to reflect value
        distances should be symmetric
        """
        total = 0
        for i in range(1, numCities+1):
            first = state[i-1]
            second = state[i]
            try:
                dist = self.map[(first, second)]
            except:
                dist = self.map[(second, first)]
        # the shortest past is the best, so distances are negated to reflect this.
            total -= dist
            if debug:
                print("current cities: ", first, second, "distance between cities: ", dist)
        return total

To test the tsp formulation, I generated 50 cities with random distances, and used hill-climbing and simulated-annealing to find a solution.

In [ ]:
print('Here\'s code to run the TSP problem')

In [ ]:
if __name__ == '__main__':
    numCities = 50
    initial = []
    for city in range(numCities):
        initial.append(city)
    initial.append(0)

    map = {}
    for cityA in range(numCities):
        for cityB in range(cityA+1, numCities):
            map[cityA, cityB] = randrange(1, 20)

    '''
    map = {(0,1): 1.5, (0,2): 2, (0,3): 3, (0,4): 2.5, (0,5): 3,
           (1,2): 7, (1,3): 2.4, (1,4): 4, (1,5): 3.6,
           (2,3): 5,(2,4): 4.5, (2,5): 1.5,
           (3,4): 2.5, (3,5): 3,
           (4,5): 7.5}
    '''

    if debug:
        print("testing the constructed map: ", map)

    tsp = TravelingSalesman(numCities, initial, map)
    print("Traveling salesman problem: ",
          "\ninitial circuit: ", initial,
          "initial circuit distance: ", -tsp.value(initial))

    hill_solution = hill_climbing(tsp)
    print("Hill-climbing solution: ", str(hill_solution), "distance: ", str(-tsp.value(hill_solution)))

    annealing_solution = simulated_annealing(tsp,
        exp_schedule(k=20, lam=0.005, limit=10000)
    )
    print("Simulated solution: ", str(annealing_solution), "distance: ", str(-tsp.value(annealing_solution)))


With this formulation of tsp, hill-climbing and simulated annealing both finds solutions that are much shorter in distance than the initial path. Their solutions are usually pretty comparable - sometimes hill-climbing finds a better one, sometimes simulated-annealing does. I did notice that simulated annealing takes much longer to run especially on the larger configurations. With the exception ofextreme scenarios where being willing to travel to further away cities will lead to saving great distances later on in the path, I think in traveling salesman problem simulated annealing probably don't have much advantage over simple hill-climbing especially after weighing the costs. It makes sense to always pick your next states that is more advantageous (shorter distance).

In [ ]:
from csp import CSP, parse_neighbors, min_conflicts, backtracking_search, AC3
from search import depth_first_graph_search

debug = 0
def courseScheduling(courses = 'cs108 cs112 cs212 cs214 cs232 cs344 cs352', faculty = 'dschuurman hplantinga jadams kvanderlinden', times = 'mwf800 mwf900 mwf1030 mwf230 tth1030', classrooms = 'nh253 sb382', assignments = {'cs108':['jadams', 'kvanderlinden', 'dschuurman'], 'cs344':['kvanderlinden'], 'cs352':['hplantinga']}):
    """
    :param courses: will be the variables in CSP formulation
    :param faculty:
    :param times:
    :param classrooms:
    :param assignments: a dict containing professors who are assigned to a course. Not all courses have assigned professors
    :return: a CSP instance with variables populated with course numbers, domains populated with any possible combinations of prof+time+location for each course, neighbors populated with any course pairings, and constraints see comments below
    """
    variables = courses.split()
    faculty = faculty.split()
    times = times.split()
    classrooms = classrooms.split()
    assignments = assignments
    domains = {}
    neighbors = {}

    def findAssignments(faculty=faculty, times = times, classrooms = classrooms):
        possibilities = []
        for prof in faculty:
            for time in times:
                for room in classrooms:
                    possibilities.append(prof+' '+time+' '+room)
        return possibilities

    for course in variables:
        neighbors[course] = [others for others in variables if others != course]
        domains[course] = findAssignments()

    if debug:
        print("testing variables: ", str(variables))
        print("testing domain: ", str(domains))
        print("testing neighbors: ", str(neighbors))

    def scheduling_constraint(A, a, B, b):
        """
        :param A: variable A (a course)
        :param a: the current assigned values for A in domain
        :param B: variable B (a course)
        :param b: the current assigned values for B in domain
        :return: true if all physical constraints and preferred prof-course pairings are satisfied
        no professor teaches two courses at the same time.
        no two courses share the same location at the same time.
        """
        profA = a.split()[0]
        profB = b.split()[0]
        tA = a.split()[1]
        tB = b.split()[1]
        rA = a.split()[2]
        rB = b.split()[2]
        if A in assignments and profA not in assignments[A]:
            return False
        elif B in assignments and profB not in assignments[B]:
            return False
        elif tA == tB:
            if profA == profB or rA == rB:
                return False
            else:
                return True
        else:
            return True


    return CSP(variables, domains, neighbors, scheduling_constraint)

In [ ]:
if __name__ == '__main__':
    '''
    tried to solve scheduling problem with AC3, backtracking, and min_conflicts.
    Mirrored printing of results from lab03/queens.py
    '''
    problem = courseScheduling()
    #solution = AC3(problem)
    #solution = backtracking_search(problem)
    solution = min_conflicts(problem)
    if type(solution) is bool:
        if solution and problem.goal_test(problem.infer_assignment()):
            print('AC3 Solution:')
        else:
            print('AC Failure:')
        print(problem.curr_domains)

    # Handle other solutions next.
    elif solution != None and problem.goal_test(solution):
        print('Solution:')
        #i used .format command as seen from this page: https://stackoverflow.com/questions/10837017/how-do-i-make-a-fixed-size-formatted-string-in-python
        print('Course:                  Professor:               Time:                Location:       ')
        for course in solution:
            prof =solution[course].split()[0]
            time = solution[course].split()[1]
            location = solution[course].split()[2]
            print('{:25s}{:25s}{:25s}{:25s}'.format(course,prof,time,location))
    #    problem.display(problem.infer_assignment())
    else:
        print('Failed - domains: ' + str(problem.curr_domains))
        problem.display(problem.infer_assignment())